In [1]:
import os
import pandas as pd
import MeCab
import faiss
import torch
from tqdm import tqdm
from glob import glob
from ast import literal_eval
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
# Mecab
mecab = MeCab.Tagger()
def extract_nouns(text):
    try:
        parsed = mecab.parse(text)
        nouns = []
        for line in parsed.splitlines():
            if '\t' in line:  # MeCab 출력에서 유효한 줄만 처리
                word, feature = line.split('\t')
                if feature.startswith('NNG') or feature.startswith('NNP'):  # 보통명사, 고유명사
                    nouns.append(word)
        return nouns
    except Exception as e:
        print(f"Error during MeCab parsing: {e}")
        return text  # 실패 시 원문 반환

In [3]:
korean_history_term_merged = pd.read_csv('../../contest_baseline_code/data/rag/korean_history_term_merged.csv')
korean_history_textbook_25_merged = pd.read_csv('../../contest_baseline_code/data/rag/korean_history_textbook_25_merged.csv')
korean_webtext_edu = pd.read_csv('../../contest_baseline_code/data/rag/korean-webtext-edu.csv')
korean_wikipedia_edu = pd.read_csv('../../contest_baseline_code/data/rag/korean-wikipedia-edu.csv')
openstax_econ = pd.read_csv('../../contest_baseline_code/data/rag/openstax_econ.csv')
openstax_poli = pd.read_csv('../../contest_baseline_code/data/rag/openstax_poli.csv')
openstax_psych = pd.read_csv('../../contest_baseline_code/data/rag/openstax_psych.csv')
openstax_us_hist = pd.read_csv('../../contest_baseline_code/data/rag/openstax_us_hist.csv')
openstax_world_hist1 = pd.read_csv('../../contest_baseline_code/data/rag/openstax_world_hist1.csv')
openstax_world_hist2 = pd.read_csv('../../contest_baseline_code/data/rag/openstax_world_hist2.csv')
train_aug_qa_cleaned = pd.read_csv('../../contest_baseline_code/data/rag/train_aug_qa_cleaned.csv')

print(korean_history_term_merged.shape)
print(korean_history_textbook_25_merged.shape)
print(korean_webtext_edu.shape)
print(korean_wikipedia_edu.shape)
print(openstax_econ.shape)
print(openstax_poli.shape)
print(openstax_psych.shape)
print(openstax_us_hist.shape)
print(openstax_world_hist1.shape)
print(openstax_world_hist2.shape)
print(train_aug_qa_cleaned.shape)

(793, 3)
(276, 3)
(7807, 3)
(1672, 3)
(2303, 3)
(2137, 3)
(2432, 3)
(3225, 3)
(2231, 3)
(2315, 3)
(818, 3)


In [4]:
######################
## RAG Data Loading ##
######################
rag_folder = "../../contest_baseline_code/data/rag"
rag_files = glob(f"{rag_folder}/*.csv")

# Concatenate RAG data
rag_data_source = [pd.read_csv(file) for file in rag_files]
rag_data = pd.concat(rag_data_source, axis=0, ignore_index=True)
print(f"RAG Data Count: {rag_data.shape[0]}")

# Use only documents with at least 25 characters
rag_data = rag_data[rag_data.context.str.len() >= 25]
print(f"filtered (len > 25) RAG Data Count: {rag_data.shape[0]}")

RAG Data Count: 26009
filtered (len > 25) RAG Data Count: 25548


In [5]:
#######################
## Rag Data Chunking ##
#######################
loader = DataFrameLoader(rag_data, page_content_column='context')
documents = loader.load()

# Chunking
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator=". ",
    chunk_size=600,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)
split_docs = text_splitter.split_documents(tqdm(documents))
print(f"Chunked Document Count: {len(split_docs)}")

100%|██████████| 25548/25548 [00:00<00:00, 699342.65it/s]
Created a chunk of size 619, which is longer than the specified 600
Created a chunk of size 1172, which is longer than the specified 600
Created a chunk of size 1113, which is longer than the specified 600
Created a chunk of size 666, which is longer than the specified 600
Created a chunk of size 3822, which is longer than the specified 600
Created a chunk of size 676, which is longer than the specified 600
Created a chunk of size 649, which is longer than the specified 600
Created a chunk of size 1005, which is longer than the specified 600
Created a chunk of size 1105, which is longer than the specified 600
Created a chunk of size 863, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 846, which is longer than the specified 600
Created a chunk of size 641, which is longer than the specified 600
Created a chunk of size 798, which is longer than the

Chunked Document Count: 81077


In [6]:
###############
## Vector DB ##
###############
model_name = 'dragonkue/BGE-m3-ko'
device = 'cuda'

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# Vector DB path
vector_store_path = f"/data/ephemeral/home/workspace/contest_baseline_code/data/db/faiss_{model_name}_count-{len(split_docs)}"

# Load existing vector store or create new one
if os.path.exists(vector_store_path):
    print("Loading existing vector store...")
    vector_store = FAISS.load_local(
        vector_store_path,
        embeddings,
        allow_dangerous_deserialization=True
        )
else:
    print("Creating new vector store...")
    vector_store = FAISS.from_documents(
        [split_docs[0]],
        embedding=embeddings,
        distance_strategy=DistanceStrategy.COSINE
    )

    # # 인덱스를 GPU로 이동
    # res = faiss.StandardGpuResources()
    # gpu_index = faiss.index_cpu_to_gpu(res, 0, vector_store.index)  # 0은 GPU ID
    # vector_store.index = gpu_index    
    
    # 배치 크기 설정
    batch_size = 4  # 적절한 배치 크기로 조절 가능
    docs_to_add = split_docs[1:]
    with tqdm(total=len(docs_to_add), desc="Ingesting documents") as pbar:
        for i in range(0, len(docs_to_add), batch_size):
            batch_docs = docs_to_add[i:i + batch_size]
            vector_store.add_documents(batch_docs)
            pbar.update(len(batch_docs))
            torch.cuda.empty_cache()
            
    # # 인덱스를 다시 CPU로 이동하여 저장
    # cpu_index = faiss.index_gpu_to_cpu(vector_store.index)
    # vector_store.index = cpu_index
    
    # Save vector DB
    vector_store.save_local(vector_store_path)

# Check the number of documents in the vector store
doc_count = vector_store.index.ntotal
print(f"Document Count in Vector Store: {doc_count}")

Creating new vector store...


Ingesting documents: 100%|██████████| 81076/81076 [33:34<00:00, 40.26it/s] 


Document Count in Vector Store: 81077


In [7]:
#######################
## Define Retrievers ##
#######################

# Set top k for retrievers
topk=5
rerank_topk = 2

# Sparse retriever
bm25_retriever = BM25Retriever.from_documents(
    documents=split_docs,
    k=topk,
    preprocess_func=extract_nouns,  # MeCab 전처리 함수
    # metadata={"source": "faq", "version": "1.0"}, # 추가 메타데이터 정보(원본 데이터 뭔지) 출력 가능
    # vectorizer=, # 커스텀 벡터라이저 사용 가능
)

# Dense retriever
faiss_retriever = vector_store.as_retriever(search_kwargs={"k":topk})

# Ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5])

# Reranker
reranker = HuggingFaceCrossEncoder(
    model_name="dragonkue/bge-reranker-v2-m3-ko",
    model_kwargs={'device': 'cuda'}
)

# Compressor
compressor = CrossEncoderReranker(model=reranker, top_n=rerank_topk)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [8]:
##############################
## Save Retrieval Documents ##
##############################
tqdm.pandas()

target_data = pd.read_csv("../../contest_baseline_code/data/raw/test.csv")
prompt = "{paragraph} {question} {choices}"

def process_row(row):
    problems = literal_eval(row['problems'])
    paragraph = row['paragraph']
    question = problems['question']
    choices = problems['choices']
    choices_str = " ".join(choices)
    
    query = prompt.format(paragraph=paragraph, question=question, choices=choices_str)
    
    if len(paragraph) > 500: # 보수적 기준: 약 600~700자, 널널한 기준: 약 300~400자
        return None
    docs = compression_retriever.invoke(query)
    retrieved_docs = [doc.page_content for doc in docs]
    return retrieved_docs
    
# query 생성
target_data['reference'] = target_data.progress_apply(process_row, axis=1)

# 결과 저장
target_data.to_csv("../../contest_baseline_code/data/preprocessed/test_rag.csv", index=False)


100%|██████████| 869/869 [15:53<00:00,  1.10s/it]
